In [ ]:
from evo.notebooks import ServiceManagerWidget
from evo.objects import ObjectAPIClient

cache_location = "data"

# Evo app credentials
client_id = "<your-client-id>"  # Replace with your client ID
redirect_url = "<your-redirect-url>"  # Replace with your redirect URL

manager = await ServiceManagerWidget.with_auth_code(
    discovery_url="https://discover.api.seequent.com",
    redirect_url=redirect_url,
    client_id=client_id,
    cache_location=cache_location,
).login()

### Use the Evo Python SDK to create an object client and a data client

In [ ]:
# The object client will manage your auth token and Geoscience Object API requests.
object_client = ObjectAPIClient(manager.get_environment(), manager.get_connector())

# The data client will manage saving your data as Parquet and publishing your data to Evo storage.
data_client = object_client.get_data_client(manager.cache)

#### Use the Object Selector Widget

The widget provides:
- A refresh button to load all objects from the workspace
- A dropdown to filter objects by type (schema classification)
- A dropdown to select a specific object
- An information panel showing details about the selected object

In [ ]:
from workspace_utils import ObjectSelectorWidget

# Create and display the object selector widget (auto-loads objects)
object_selector = ObjectSelectorWidget(object_client)
await object_selector.display()

In [ ]:
# Get the selected object ID and print it
selected_object = object_selector.get_selected_object()
selected_id = object_selector.get_selected_id()

if selected_id:
    print(f"Selected Object ID: {selected_id}")
    print(f"Selected Object Name: {object_selector._format_object_name(selected_object)}")
else:
    print("No object selected. Please select an object from the dropdown above.")

#### Select Target Workspace and Operation

This widget allows you to:
- Select a target workspace to copy or move the object to
- Check "Delete original after copy" to perform a move operation instead of copy
- View details about the selected target workspace

In [ ]:
from workspace_utils import TargetWorkspaceSelectorWidget

# Create and display the target workspace selector widget (auto-loads workspaces)
target_workspace_selector = TargetWorkspaceSelectorWidget(manager)
await target_workspace_selector.display()

#### Download and Save Object

Download the selected object's JSON and save the "object" portion to the data folder.

In [ ]:
import json
from pathlib import Path

from workspace_utils import collect_data_references

from evo.notebooks import FeedbackWidget

# Download the selected object
if selected_id:
    # Download the object
    downloaded_object = await object_client.download_object_by_id(object_id=selected_id)

    # Get the object as a dictionary
    object_data = downloaded_object.as_dict()

    # Collect all data references from the object
    data_refs = collect_data_references(object_data)

    if data_refs:
        print(f"Found {len(data_refs)} data file(s) to download...")

        # Download all data files to the cache
        for data_download in downloaded_object.prepare_data_download(list(data_refs)):
            await data_download.download_to_cache(
                cache=manager.cache,
                transport=object_client._connector.transport,
                fb=FeedbackWidget(f"Downloading {data_download.name}"),
            )
        print("All data files downloaded to cache.")
    else:
        print("No data files to download.")

    # Remove only the top-level 'uuid' field if it exists
    object_data.pop("uuid", None)

    # Create data folder if it doesn't exist
    data_folder = Path("data")
    data_folder.mkdir(exist_ok=True)

    # Save to file using the object name
    object_name = object_selector._format_object_name(selected_object)
    output_file = data_folder / f"{object_name}.json"

    with open(output_file, "w") as f:
        json.dump(object_data, f, indent=2, default=str)

    print(f"Object data saved to: {output_file}")
    print(f"File size: {output_file.stat().st_size} bytes")
else:
    print("No object selected. Please select an object first.")

#### Publish Object to Target Workspace

Read the saved JSON file and publish it as a geoscience object to the selected target workspace.

In [ ]:
import json
from pathlib import Path

from evo.common import Environment
from evo.notebooks import FeedbackWidget

# Get the target workspace ID and operation
target_workspace_id = target_workspace_selector.get_selected_workspace_id()
operation = target_workspace_selector.get_operation()

if not target_workspace_id:
    print("No target workspace selected. Please select a target workspace first.")
elif not selected_id:
    print("No object selected. Please select an object first.")
else:
    # Read the saved JSON file
    object_name = object_selector._format_object_name(selected_object)
    input_file = Path("data") / f"{object_name}.json"

    if not input_file.exists():
        print(f"Error: File {input_file} not found. Please run the download cell first.")
    else:
        # Read the JSON data
        with open(input_file, "r") as f:
            object_data = json.load(f)

        # Publish the object to the target workspace
        operation_verb = "Copying" if operation == "Copy" else "Moving"
        print(f"{operation_verb} object '{object_name}' to target workspace...")

        try:
            # Create a new ObjectAPIClient with the target workspace environment
            target_env = Environment(
                org_id=manager.get_environment().org_id,
                workspace_id=target_workspace_id,
                hub_url=manager.get_environment().hub_url,
            )

            target_object_client = ObjectAPIClient(target_env, manager.get_connector())

            # Use the same cache as the source object client
            # This ensures the data files downloaded earlier are accessible
            target_data_client = target_object_client.get_data_client(manager.cache)

            # Copy the data files from the source workspace cache to the target workspace cache
            # Get cache locations for both workspaces
            source_cache_location = data_client.cache_location
            target_cache_location = target_data_client.cache_location

            # Collect all data references from the object
            data_refs = collect_data_references(object_data)

            if source_cache_location != target_cache_location and data_refs:
                print(f"Copying {len(data_refs)} data file(s) to target workspace cache...")
                import shutil

                target_cache_location.mkdir(parents=True, exist_ok=True)

                for data_ref in data_refs:
                    source_file = source_cache_location / data_ref
                    target_file = target_cache_location / data_ref
                    if source_file.exists() and not target_file.exists():
                        shutil.copy2(source_file, target_file)
                print("Data files copied to target workspace cache.")

            # Upload the referenced data to the target workspace
            await target_data_client.upload_referenced_data(object_data, FeedbackWidget("Uploading data"))

            # Create the geoscience object in the target workspace
            # Use the same path as the original object
            target_path = selected_object.path
            published_object = await target_object_client.create_geoscience_object(target_path, object_data)

            print("Successfully published object to target workspace!")
            print(f"New Object ID: {published_object.id}")
            print(f"New Object Path: {published_object.path}")

            # If this is a move operation, delete the original object
            if operation == "Move":
                print("Deleting original object from source workspace...")
                await object_client.delete_object_by_id(selected_id)
                print("Original object deleted successfully.")

        except Exception as e:
            print(f"Error publishing object: {str(e)}")